<a href="https://colab.research.google.com/github/tomoyahiroe/transformers-playground/blob/main/how_tokenizer_insert_special_token.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>